In [1]:
import pickle as pkl
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score, precision_score, recall_score

import sys
sys.path.insert(1, '../')
from encoding import prefix_bin
import numpy as np
import json
import os

In [2]:
def trueTo1(x):
    y_true = []
    for i in x:
        if i =='True':
            y_true.append(1)
        else:
            y_true.append(0)
    return y_true

In [3]:
def return_no_0(y, evaluationlist):
    y_0 =[]
    eval_0=[]
    
    for pos, i in enumerate(evaluationlist):
        if i != 0.0:
            y_0.append(y[pos])
            eval_0.append(i)
            
    return y_0, eval_0

In [9]:
evaluation_dict = {}
result_dict = {}
result_no_avg_dict = {}
evaluation = 'aggregated'
dataset_label = 'OIR5K'
training_rebalance = True
coverage_df = {}

for classifier in ['hatc']:
    with open('../dataset_parameters.json','r') as json_file:
        parameters = json.load(json_file)[dataset_label]
        print(parameters)
        key_pair = parameters['key_pair']
        catatars = parameters['categorical_attrs']
        maximum_prefixs = parameters['maximum_prefix']
        

    for prefix in range(1,13):
        print(prefix)
        evaluation_dict[prefix] = {}
        result_dict[prefix] = {}

        training_data = '../result/%s/%s/Finished cases/train_rebalance_%s prefix_%s training window retrained.pkl'%(dataset_label, classifier, training_rebalance, prefix)
        test_data = '../result/%s/%s/Finished cases/train_rebalance_%s prefix_%s test window retrained.pkl'%(dataset_label, classifier, training_rebalance, prefix)
        result = '../result/%s/%s/Finished cases/train_rebalance_%s prefix_%s update retrained.pkl'%(dataset_label, classifier, training_rebalance, prefix)

        with open(training_data, 'rb') as openfile:
            training_data = pkl.load(openfile)

        with open(test_data, 'rb') as openfile:
            test_data = pkl.load(openfile)

        with open(result, 'rb') as openfile:
            result = pkl.load(openfile)

        training_label = {'True':0, 'False':0}
        training_data = training_data[200][0]
    #     if classifier == 'hatc':
    #         training_data = training_data[200]
    #     else:
    #         training_data = training_data[200][0]
        for t in training_data:
            label = t.true_label
            training_label[label] +=1

        testing_label={}
        for updates in test_data.keys():
            testing_label[updates] = {'True':0, 'False':0}
            for t in test_data[updates]:
                label = t.true_label
                testing_label[updates][label]+=1

        evaluationlist = {'fscore':[], 'precision':[], 'recall':[]}


        for updates in result[prefix].keys():
            if classifier == 'hatc':
                y_pred = [max(i, key=i.get) for i in result[prefix][updates]['y_pred']]
                y_pred = trueTo1(y_pred)
            else:
                y_pred = [np.argmax(i[0]) for i in result[prefix][updates]['y_pred']]

            y_true = trueTo1(result[prefix][updates]['y_true'])

            evaluationlist['fscore'].append(f1_score(y_true = y_true, y_pred = y_pred, average='weighted'))
            evaluationlist['precision'].append(precision_score(y_true = y_true, y_pred = y_pred, average='weighted'))
            evaluationlist['recall'].append(recall_score(y_true = y_true, y_pred = y_pred, average='weighted'))

        df = pd.DataFrame.from_dict(testing_label).T
        y = [i/30 for i in list(df['True'])]

        for i in evaluationlist:
            y_, eval_ = return_no_0(y, evaluationlist[i])
            y_delta = [y_[pos]-y[pos-1] for pos in range(len(y_)) if pos !=0]
            eval_delta = [eval_[pos]-eval_[pos-1] for pos in range(len(eval_)) if pos !=0]
            result_dict[prefix][i] = np.average(evaluationlist[i])
            evaluation_dict[prefix][i] = np.corrcoef(y_, eval_)[0][1]
            
        coverage_df[prefix] = y

        try:
            os.makedirs('../result/%s/%s/performance'%(dataset_label, classifier))
        except:
            pass
        
        perf_df = pd.DataFrame.from_dict(evaluationlist)
        perf_df_dir = '../result/%s/%s/performance/rebalanced_%s prefix_%s performance retrained.csv'%(dataset_label, classifier, str(training_rebalance), str(prefix))
        perf_df.to_csv(perf_df_dir, index=False)

    coverage_df = pd.DataFrame.from_dict(coverage_df)
    
    eval_df = pd.DataFrame.from_dict(evaluation_dict)
    result_df = pd.DataFrame.from_dict(result_dict)
    
    coverage_df_dir = '../result/%s/%s/rebalanced_%s label retrained.csv'%(dataset_label, classifier, str(training_rebalance))
    coverage_df.to_csv(coverage_df_dir, index=False)

    eval_df_dir = '../result/%s/%s/rebalanced_%s label corr retrained.pkl'%(dataset_label, classifier, str(training_rebalance))
    result_df_dir = '../result/%s/%s/rebalanced_%s perf retrained.pkl'%(dataset_label, classifier, str(training_rebalance))
    with open(eval_df_dir, 'wb') as r:
        pkl.dump(eval_df, r)

    with open(result_df_dir, 'wb') as r:
        pkl.dump(result_df, r)

{'key_pair': {'Case ID': 'caseid', 'Activity': 'activity', 'Complete Timestamp': 'ts'}, 'categorical_attrs': ['activity'], 'maximum_prefix': 12}
1


c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 

c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 

c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 

c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 

...

c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


3
4
5
6
7
8
9
10
11


c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


12


c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
pd.DataFrame.from_dict(evaluationlist)

,fscore,precision,recall
0,1.000000,1.000000,1.000000
1,1.000000,1.000000,1.000000
2,1.000000,1.000000,1.000000
3,1.000000,1.000000,1.000000
4,1.000000,1.000000,1.000000
...,...,...,...
1159,0.933937,0.942857,0.933333
1160,0.933937,0.942857,0.933333
1161,0.933937,0.942857,0.933333
1162,0.933937,0.942857,0.933333
